In [4]:
# Ensure the project root is in the Python path
import sys
sys.path.append('/Users/tl759k/Documents/GitHub/work/cursor-analytics')

# Activate the virtual environment (if not already activated)
# This is typically done outside the notebook, but we ensure the path is correct here
import os
os.system('source /Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/bin/activate')

# Re-import the SnowflakeHook after ensuring the path is set
from utils.snowflake_connection import SnowflakeHook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import plotly.graph_objects as go
import seaborn as sns
import statsmodels.formula.api as smf
from decimal import Decimal
import plotly.express as px


# Load and execute SQL queries for data jobs (no data return)
def execute_sql_job(file_path):
    """
    Execute SQL statements from a file for table refresh/creation jobs.
    Handles multiple statements (CREATE TABLE, GRANT, etc.) without returning data.
    
    Args:
        file_path (str): Path to SQL file containing statements
        
    Returns:
        bool: True if all statements executed successfully
    """
    import time
    from datetime import datetime, timedelta
    
    # Start overall timer
    overall_start_time = time.time()
    start_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"🔄 Starting SQL job from: {file_path}")
    print(f"⏰ Start time: {start_timestamp}")
    
    # Read SQL file
    with open(file_path, 'r') as file:
        query_content = file.read()
    
    # Initialize Snowflake connection with larger warehouse for better performance
    connection_start = time.time()
    snowhook = SnowflakeHook(
        warehouse='DCR_WH_4XLARGE'  # Use 4XL warehouse for fastest performance
    )
    connection_time = time.time() - connection_start
    print(f"✅ Connected to Snowflake with DCR_WH_4XLARGE warehouse (took {connection_time:.2f}s)")
    
    # Split SQL statements by semicolon and execute each one
    statements = [stmt.strip() for stmt in query_content.split(';') if stmt.strip()]
    total_statements = len(statements)
    
    print(f"📋 Found {total_statements} SQL statement(s) to execute")
    print("=" * 60)
    
    statement_times = []
    
    try:
        for i, statement in enumerate(statements, 1):
            # Calculate progress
            progress_pct = ((i-1) / total_statements) * 100
            
            # Estimate time remaining based on average of completed statements
            if len(statement_times) > 0:
                avg_time = sum(statement_times) / len(statement_times)
                remaining_statements = total_statements - (i-1)
                estimated_remaining = avg_time * remaining_statements
                eta_str = f" | ETA: {estimated_remaining:.1f}s"
            else:
                eta_str = ""
            
            print(f"🔄 [{progress_pct:5.1f}%] Statement {i}/{total_statements}{eta_str}")
            print(f"   📝 Preview: {statement[:80]}{'...' if len(statement) > 80 else ''}")
            
            # Execute statement with timing
            stmt_start_time = time.time()
            snowhook.query_without_result(statement)
            stmt_duration = time.time() - stmt_start_time
            statement_times.append(stmt_duration)
            
            # Update progress
            progress_pct = (i / total_statements) * 100
            print(f"   ✅ Completed in {stmt_duration:.2f}s [{progress_pct:5.1f}%]")
            print("-" * 60)
        
        # Final timing summary
        total_duration = time.time() - overall_start_time
        end_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print(f"🎉 All {total_statements} statements executed successfully!")
        print("=" * 60)
        print(f"📊 TIMING SUMMARY:")
        print(f"   • Start time: {start_timestamp}")
        print(f"   • End time: {end_timestamp}")
        print(f"   • Total duration: {total_duration:.2f}s ({total_duration/60:.2f} minutes)")
        print(f"   • Connection time: {connection_time:.2f}s")
        print(f"   • Average per statement: {sum(statement_times)/len(statement_times):.2f}s")
        print(f"   • Fastest statement: {min(statement_times):.2f}s")
        print(f"   • Slowest statement: {max(statement_times):.2f}s")
        
        return True
        
    except Exception as e:
        total_duration = time.time() - overall_start_time
        end_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print(f"❌ Error executing SQL job: {str(e)}")
        print("=" * 60)
        print(f"⚠️ JOB FAILED AFTER {total_duration:.2f}s")
        print(f"   • Failed at statement {i}/{total_statements}")
        print(f"   • Completed statements: {i-1}")
        if statement_times:
            print(f"   • Average time per completed statement: {sum(statement_times)/len(statement_times):.2f}s")
        return False


# Keep the original function for data retrieval (single statement only)
def load_query(file_path):
    """
    Load data from a single SQL SELECT statement.
    Use this for queries that return data you want to analyze.
    
    Args:
        file_path (str): Path to SQL file with single SELECT statement
        
    Returns:
        pd.DataFrame: Query results as DataFrame
    """
    with open(file_path, 'r') as file:
        query = file.read()
    
    snowhook = SnowflakeHook(
        warehouse='DCR_WH_4XLARGE'  # Use 4XL warehouse for better performance
    )
    df = snowhook.query_snowflake(query, method='pandas')

    # Format decimal to float
    for col in df.columns:
        if df[col].dtype == 'object':
            # Check if column has any non-null values before checking type
            non_null_values = df[col].dropna()
            if len(non_null_values) > 0 and isinstance(non_null_values.iloc[0], Decimal):
                df[col] = df[col].astype(float)

    return df

In [5]:
# Helper function to create a visual progress bar
def create_progress_bar(percentage, width=30):
    """Create a visual progress bar string"""
    filled = int(width * percentage / 100)
    bar = "█" * filled + "░" * (width - filled)
    return f"[{bar}] {percentage:5.1f}%"

# Enhanced version with visual progress bar
def execute_sql_job_with_progress_bar(file_path):
    """
    Enhanced version with visual progress bar for better monitoring
    """
    import time
    from datetime import datetime
    
    # Start overall timer
    overall_start_time = time.time()
    start_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # print(f"🔄 Starting SQL job from: {file_path}")
    # print(f"⏰ Start time: {start_timestamp}")
    
    # Read SQL file
    with open(file_path, 'r') as file:
        query_content = file.read()
    
    # Initialize Snowflake connection
    connection_start = time.time()
    snowhook = SnowflakeHook()
    connection_time = time.time() - connection_start
    # print(f"✅ Connected to Snowflake (took {connection_time:.2f}s)")
    
    # Split SQL statements
    statements = [stmt.strip() for stmt in query_content.split(';') if stmt.strip()]
    total_statements = len(statements)
    
    # print(f"📋 Found {total_statements} SQL statement(s) to execute")
    # print("=" * 70)
    
    statement_times = []
    
    try:
        for i, statement in enumerate(statements, 1):
            # Calculate progress
            progress_pct = ((i-1) / total_statements) * 100
            
            # Create visual progress bar
            progress_bar = create_progress_bar(progress_pct)
            
            # Estimate time remaining
            if len(statement_times) > 0:
                avg_time = sum(statement_times) / len(statement_times)
                remaining_statements = total_statements - (i-1)
                estimated_remaining = avg_time * remaining_statements
                eta_str = f"ETA: {estimated_remaining:.1f}s"
            else:
                eta_str = "ETA: calculating..."
            
            # print(f"\n🔄 Statement {i}/{total_statements} | {eta_str}")
            # print(f"   {progress_bar}")
            # print(f"   📝 {statement[:60]}{'...' if len(statement) > 60 else ''}")
            
            # Execute statement with timing
            stmt_start_time = time.time()
            snowhook.query_without_result(statement)
            stmt_duration = time.time() - stmt_start_time
            statement_times.append(stmt_duration)
            
            # Update progress
            progress_pct = (i / total_statements) * 100
            progress_bar_complete = create_progress_bar(progress_pct)
            
            print(f"   ✅ Completed in {stmt_duration:.2f}s")
            print(f"   {progress_bar_complete}")
        
        # Final summary
        total_duration = time.time() - overall_start_time
        end_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print("\n" + "=" * 70)
        print(f"🎉 ALL STATEMENTS COMPLETED SUCCESSFULLY!")
        print(f"📊 PERFORMANCE SUMMARY:")
        print(f"   ⏰ Duration: {total_duration:.2f}s ({total_duration/60:.2f} min)")
        print(f"   🔗 Connection: {connection_time:.2f}s")
        print(f"   📊 Avg/statement: {sum(statement_times)/len(statement_times):.2f}s")
        print(f"   ⚡ Fastest: {min(statement_times):.2f}s")
        print(f"   🐌 Slowest: {max(statement_times):.2f}s")
        
        return True
        
    except Exception as e:
        total_duration = time.time() - overall_start_time
        print(f"\n❌ ERROR: {str(e)}")
        print(f"⚠️ Failed after {total_duration:.2f}s at statement {i}/{total_statements}")
        return False

print("📦 Enhanced SQL job functions loaded!")
print("💡 Use execute_sql_job_with_progress_bar('test.sql') for visual progress tracking")


📦 Enhanced SQL job functions loaded!
💡 Use execute_sql_job_with_progress_bar('test.sql') for visual progress tracking


In [6]:
step_0 = execute_sql_job_with_progress_bar('s0_tbl_applicant_funnel_timestamp_with_backfill_snowflake.sql')

2025-09-22 05:21:39,907 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-22 05:21:39,907 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


2025-09-22 05:21:40,694 - utils.snowflake_connection - INFO - Successfully connected to Snowflake
   ✅ Completed in 456.45s
   [███████████████░░░░░░░░░░░░░░░]  50.0%
   ✅ Completed in 0.16s
   [██████████████████████████████] 100.0%

🎉 ALL STATEMENTS COMPLETED SUCCESSFULLY!
📊 PERFORMANCE SUMMARY:
   ⏰ Duration: 456.73s (7.61 min)
   🔗 Connection: 0.12s
   📊 Avg/statement: 228.31s
   ⚡ Fastest: 0.16s
   🐌 Slowest: 456.45s


In [7]:
step_1 = execute_sql_job_with_progress_bar('s1_tbl_major_steps_conversion_analysis_applied_L7D_cohort_snowflake.sql')


2025-09-22 05:29:16,803 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-22 05:29:16,804 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


2025-09-22 05:29:17,657 - utils.snowflake_connection - INFO - Successfully connected to Snowflake
   ✅ Completed in 209.53s
   [███████████████░░░░░░░░░░░░░░░]  50.0%
   ✅ Completed in 0.17s
   [██████████████████████████████] 100.0%

🎉 ALL STATEMENTS COMPLETED SUCCESSFULLY!
📊 PERFORMANCE SUMMARY:
   ⏰ Duration: 209.96s (3.50 min)
   🔗 Connection: 0.26s
   📊 Avg/statement: 104.85s
   ⚡ Fastest: 0.17s
   🐌 Slowest: 209.53s


In [8]:
step_2 = execute_sql_job_with_progress_bar('s2_tbl_cvr_reporting_metric_variances_snowflake.sql')


2025-09-22 05:32:46,645 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-22 05:32:46,646 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


2025-09-22 05:32:47,239 - utils.snowflake_connection - INFO - Successfully connected to Snowflake
   ✅ Completed in 17.54s
   [███████████████░░░░░░░░░░░░░░░]  50.0%
   ✅ Completed in 0.17s
   [██████████████████████████████] 100.0%

🎉 ALL STATEMENTS COMPLETED SUCCESSFULLY!
📊 PERFORMANCE SUMMARY:
   ⏰ Duration: 17.84s (0.30 min)
   🔗 Connection: 0.13s
   📊 Avg/statement: 8.86s
   ⚡ Fastest: 0.17s
   🐌 Slowest: 17.54s


In [9]:
step_3 = execute_sql_job_with_progress_bar('s3_tbl_conversion_funnel_idv_substeps_all_timestamps_snowflake.sql')

2025-09-22 05:33:04,492 - utils.snowflake_connection - ERROR - Failed to create optimized Spark session: Java gateway process exited before sending its port number
2025-09-22 05:33:04,492 - utils.snowflake_connection - ERROR - Failed to create Spark session: Java gateway process exited before sending its port number


The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/tl759k/Documents/GitHub/work/cursor-analytics/venv/lib/python3.11/site-packages/pyspark/bin/spark-class: line 96: CMD: bad array subscript
head: illegal line count -- -1


2025-09-22 05:33:05,151 - utils.snowflake_connection - INFO - Successfully connected to Snowflake
   ✅ Completed in 47.11s
   [███████████████░░░░░░░░░░░░░░░]  50.0%
   ✅ Completed in 0.17s
   [██████████████████████████████] 100.0%

🎉 ALL STATEMENTS COMPLETED SUCCESSFULLY!
📊 PERFORMANCE SUMMARY:
   ⏰ Duration: 47.40s (0.79 min)
   🔗 Connection: 0.12s
   📊 Avg/statement: 23.64s
   ⚡ Fastest: 0.17s
   🐌 Slowest: 47.11s
